<a href="https://colab.research.google.com/github/drfperez/openair/blob/main/timeVariarion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Instalar paquetes si faltan (con dependencias para evitar problemas)
if (!require("openair")) install.packages("openair", dependencies = TRUE)
if (!require("tidyr"))   install.packages("tidyr")
if (!require("dplyr"))   install.packages("dplyr")
if (!require("reticulate")) install.packages("reticulate")

# Cargar librerías
library(openair)
library(tidyr)
library(dplyr)
library(reticulate)

# Leer datos (sube "data.csv" al panel Files de Colab)
data_long <- read.csv("data.csv", stringsAsFactors = FALSE)

# Limpieza robusta de la columna value
# Elimina todo lo que no sea número, punto o signo menos (unidades, texto, etc.)
data_long$value <- gsub("[^0-9.\\-]", "", data_long$value)
data_long$value <- as.numeric(data_long$value)

# Contar NA generados (para saber si hay problemas en los datos originales)
cat("Número de NA en 'value' después de conversión:", sum(is.na(data_long$value)), "\n")
cat("Total filas:", nrow(data_long), "\n")

# Convertir fecha (prueba auto; si falla, descomenta y ajusta el format)
data_long$date <- as.POSIXct(data_long$date, tz = "UTC")
# Si da error: data_long$date <- as.POSIXct(data_long$date, format = "%Y-%m-%d %H:%M:%S", tz = "UTC")

# Diagnóstico (mira esto en la salida!)
print("Estructura de los datos:")
str(data_long)
print("Primeras filas:")
print(head(data_long, 10))

# Pivot a wide (versión compatible con tidyr antigua)
data_wide <- data_long %>%
  pivot_wider(
    names_from  = pollutant,
    values_from = value,
    values_fn   = function(x) mean(x, na.rm = TRUE)
  )

# Si hay duplicados o warnings, aparecerán aquí arriba

# Generar timeVariation
pollutants <- setdiff(names(data_wide), "date")
if (length(pollutants) == 0) {
  stop("No se detectaron columnas de contaminantes. Revisa pollutant y el pivot.")
}

tv <- timeVariation(data_wide, pollutant = pollutants, plot = FALSE)

# Guardar gráficos como PNG
for (i in seq_along(tv$plot)) {
  filename <- paste0("time_variation_plot_", i, ".png")
  png(filename, width = 800, height = 600)
  print(tv$plot[[i]])
  dev.off()
  cat("Guardado:", filename, "\n")
}

# Descargar con reticulate + Python
use_python("/usr/bin/python3")
py_run_string("from google.colab import files")

for (i in seq_along(tv$plot)) {
  filename <- paste0("time_variation_plot_", i, ".png")
  py_run_string(paste0("files.download('", filename, "')"))
}

Número de NA en 'value' después de conversión: 90640 
Total filas: 1703544 
[1] "Estructura de los datos:"
'data.frame':	1703544 obs. of  3 variables:
 $ date     : POSIXct, format: "1991-11-20 00:00:00" "1991-11-20 00:00:00" ...
 $ pollutant: chr  "co" "h2s" "hcnm" "no" ...
 $ value    : num  2 2 2 6 10 NA 10 2 2 2 ...
[1] "Primeras filas:"
                  date pollutant value
1  1991-11-20 00:00:00        co     2
2  1991-11-20 00:00:00       h2s     2
3  1991-11-20 00:00:00      hcnm     2
4  1991-11-20 00:00:00        no     6
5  1991-11-20 00:00:00       no2    10
6  1991-11-20 00:00:00        o3    NA
7  1991-11-20 00:00:00       so2    10
8  1991-11-20 01:00:00        co     2
9  1991-11-20 01:00:00       h2s     2
10 1991-11-20 01:00:00      hcnm     2
